In [3]:
import matplotlib
from matplotlib import pylab
import pandas as pd

cols = ['Season', 'TeamID', 'Wins', 'Losses', 'PPG', 'RPG', 'APG', 'SPG', 'BPG', 'PFPG', 'TOPG']
years = [2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]

temp_df = pd.read_csv("./MRegularSeasonDetailedResults.csv",
                    names=['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc', 'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF'],
                    #dtype={'Season': 'str', 'DayNum': 'str', 'WTeamID': 'str', 'WScore': 'int', 'LTeamID': 'str', 'LScore': 'int', 'WLoc': 'str', 'NumOT': 'str', 'WFGM': 'int', 'WFGA': 'int', 'WFGM3': 'int', 'WFGA3': 'int', 'WFTM': 'int', 'WFTA': 'int', 'WOR': 'int', 'WDR': 'int', 'WAst': 'int', 'WTO': 'int', 'WStl': 'int', 'WBlk': 'int', 'WPF': 'int', 'LFGM': 'int', 'LFGA': 'int', 'LFGM3': 'int', 'LFGA3': 'int', 'LFTM': 'int', 'LFTA': 'int', 'LOR': 'int', 'LDR': 'int', 'LAst': 'int', 'LTO': 'int', 'LStl': 'int', 'LBlk': 'int', 'LPF': 'int'},
                    encoding = "ISO-8859-1")

tourney_temp_df = pd.read_csv("./MNCAATourneyDetailedResults.csv",
                        names=['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc', 'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF'],
                        encoding = "ISO-8859-1")

reg_season_df = temp_df.drop(labels=0, axis=0)
tourney_df = tourney_temp_df.drop(labels=0, axis=0)

team_avgs = pd.DataFrame(columns=cols)
reg_season_df['Season'] = pd.to_numeric(reg_season_df['Season'])
tourney_df['Season'] = pd.to_numeric(tourney_df['Season'])

for year in years:
    year_df = reg_season_df[reg_season_df['Season'] == year]
    tourney_year_df = tourney_df[tourney_df['Season'] == year]
    dict = {}
    for index, row in tourney_year_df.iterrows():
        #print(row['WTeamID'])
        dict[int(row['WTeamID'])] = 1
        dict[int(row['LTeamID'])] = 1
    
    for key in dict:
        team_1211W = year_df[year_df['WTeamID'].apply(pd.to_numeric) == key]
        team_1211L = year_df[year_df['LTeamID'].apply(pd.to_numeric) == key]
        
        #print(f"key: ", key)
        #print(f"year: ", year)
        #print(f"w_df: ", team_1211W)
        #print(f"l_df: ", team_1211L)
    
        team_1211_Wdf = team_1211W[['Season', 'WTeamID', 'WScore', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']].apply(pd.to_numeric)
        team_1211_Ldf = team_1211L[['Season', 'LTeamID', 'LScore', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']].apply(pd.to_numeric)


        
        w_sum = team_1211_Wdf.sum(axis=0, numeric_only=True)
        
        l_sum = {'Season': 0, 'LTeamID': 0, 'LScore': 0, 'LFGM': 0, 'LFGA': 0, 'LFGM3': 0, 'LFGA3': 0, 'LFTM': 0, 'LFTA': 0, 'LOR': 0, 'LDR': 0, 'LAst': 0, 'LTO': 0, 'LStl': 0, 'LBlk': 0, 'LPF': 0}
        if not team_1211L.empty:
            l_sum = team_1211_Ldf.sum(axis=0, numeric_only=True)
        
        tot_games = (len(team_1211_Wdf.index) + len(team_1211_Ldf.index))

        #print(f"w_sum: ", w_sum)
        #print(f"l_sum: ", l_sum)
        season_total = (w_sum['Season'] + l_sum['Season']) / tot_games

        id_total = (w_sum['WTeamID'] + l_sum['LTeamID']) / tot_games
        score_total = (w_sum['WScore'] + l_sum['LScore']) / tot_games
        rebs_total = (w_sum['WOR'] + l_sum['LOR'] + w_sum['WDR'] + l_sum['LDR']) / tot_games
        ast_total = (w_sum['WAst'] + l_sum['LAst']) / tot_games
        stl_total = (w_sum['WStl'] + l_sum['LStl']) / tot_games
        blk_total = (w_sum['WBlk'] + l_sum['LBlk']) / tot_games
        pf_total = (w_sum['WPF'] + l_sum['LPF']) / tot_games
        to_total = (w_sum['WTO'] + l_sum['LTO']) / tot_games

        d = {'Season': [season_total], 'TeamID': [id_total], 'Wins': [len(team_1211_Wdf.index)], 'Losses': [len(team_1211_Ldf.index)], 'PPG': [score_total], 'RPG': [rebs_total], 'APG': [ast_total], 'SPG': [stl_total], 'BPG': [blk_total], 'PFPG': [pf_total], 'TOPG': [to_total]}
        #temp_df = pd.DataFrame(data=d)
        team_avgs.loc[len(team_avgs.index)] = [season_total, id_total, len(team_1211_Wdf.index), len(team_1211_Ldf.index), score_total, rebs_total, ast_total, stl_total, blk_total, pf_total, to_total]
        #print(rows)

team_avgs

    

,Season,TeamID,Wins,Losses,PPG,RPG,APG,SPG,BPG,PFPG,TOPG
0,2006.0,1284.0,18.0,14.0,64.843750,32.656250,14.656250,7.656250,3.437500,16.937500,15.093750
1,2006.0,1214.0,16.0,15.0,66.806452,39.225806,12.354839,7.645161,3.064516,20.870968,16.225806
2,2006.0,1104.0,17.0,12.0,69.965517,37.000000,12.206897,6.344828,5.034483,13.827586,13.517241
3,2006.0,1266.0,19.0,10.0,74.206897,35.551724,15.275862,7.517241,3.068966,19.482759,14.793103
4,2006.0,1130.0,25.0,7.0,74.531250,35.187500,17.812500,5.937500,4.281250,16.500000,13.312500
...,...,...,...,...,...,...,...,...,...,...,...
932,2019.0,1205.0,20.0,11.0,75.774194,32.870968,13.612903,6.225806,2.483871,16.193548,11.774194
933,2019.0,1439.0,24.0,8.0,74.000000,32.593750,15.343750,6.656250,2.312500,15.281250,11.375000
934,2019.0,1387.0,23.0,12.0,67.057143,39.771429,12.971429,7.085714,4.057143,17.485714,12.714286
935,2019.0,1449.0,26.0,8.0,69.823529,31.588235,11.676471,9.000000,5.735294,18.411765,13.352941


In [5]:
# make a new empty dataframe to hold all of the data

# note: "Winner" column will always be '1' because the winning team is always listed first;
# it's included just for a classification metric for the machine learning
train_cols = ["Season", "Team1ID", "Team2ID", "Winner", "Team1W", "Team1L", "Team1PPG", "Team1RPG", "Team1APG", "Team1SPG", \
              "Team1BPG", "Team1PFPG", "Team1TOPG", "Team2W", "Team2L", "Team2PPG", "Team2RPG", "Team2APG", "Team2SPG",\
              "Team2BPG", "Team2PFPG", "Team2TOPG"]

train_DF = pd.DataFrame(columns=train_cols)

# we just need three columns from our tourney_df
trim_tourney_df = tourney_df[["Season", "WTeamID", "LTeamID"]]
trim_tourney_df

for year in years:
    avg_year_df = team_avgs[team_avgs['Season'] == year]
    tourney_year_df = trim_tourney_df[trim_tourney_df['Season'] == year].apply(pd.to_numeric)
    
    for index, row in tourney_year_df.iterrows():
        # make a list that we can insert into the new dataframe
        #first 4 values (last value is the "Winner" classification which is always 1)
        temp_list = [year, row["WTeamID"], row["LTeamID"], 1]
        team1_row = avg_year_df[avg_year_df["TeamID"] == temp_list[1]][cols[2:]].values.tolist()
        team2_row = avg_year_df[avg_year_df["TeamID"] == temp_list[2]][cols[2:]].values.tolist()
        temp_list.extend(team1_row[0])
        temp_list.extend(team2_row[0])
        
        # add that list to our DF
        temp_series = pd.Series(temp_list, index=train_cols)
        train_DF = train_DF.append(temp_series, ignore_index=True)

test_data = train_DF[train_DF['Season'] == 2019]
test_labels = test_data['Winner']
test_data = test_data.drop(columns=['Winner'])

train_data = train_DF[train_DF['Season'] != 2019]
train_labels = train_data['Winner']
train_data = train_data.drop(columns=['Winner'])


train_data
        
        

    

,Season,Team1ID,Team2ID,Team1W,Team1L,Team1PPG,Team1RPG,Team1APG,Team1SPG,Team1BPG,...,Team1TOPG,Team2W,Team2L,Team2PPG,Team2RPG,Team2APG,Team2SPG,Team2BPG,Team2PFPG,Team2TOPG
0,2006.0,1284.0,1214.0,18.0,14.0,64.843750,32.656250,14.656250,7.656250,3.437500,...,15.093750,16.0,15.0,66.806452,39.225806,12.354839,7.645161,3.064516,20.870968,16.225806
1,2006.0,1104.0,1266.0,17.0,12.0,69.965517,37.000000,12.206897,6.344828,5.034483,...,13.517241,19.0,10.0,74.206897,35.551724,15.275862,7.517241,3.068966,19.482759,14.793103
2,2006.0,1130.0,1334.0,25.0,7.0,74.531250,35.187500,17.812500,5.937500,4.281250,...,13.312500,22.0,7.0,73.724138,35.172414,15.655172,5.931034,2.103448,15.379310,12.551724
3,2006.0,1181.0,1380.0,30.0,3.0,82.515152,32.909091,15.363636,9.545455,5.424242,...,13.393939,19.0,12.0,65.774194,38.806452,12.354839,7.580645,3.806452,17.870968,17.580645
4,2006.0,1196.0,1375.0,27.0,6.0,79.272727,35.242424,17.030303,8.030303,4.939394,...,14.878788,23.0,6.0,74.103448,33.931034,14.172414,8.482759,1.310345,18.724138,13.241379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
851,2018.0,1242.0,1181.0,27.0,7.0,81.500000,35.205882,17.000000,6.617647,4.176471,...,11.676471,26.0,7.0,84.696970,41.878788,17.515152,7.515152,5.363636,15.636364,12.606061
852,2018.0,1437.0,1403.0,30.0,4.0,87.058824,35.588235,16.676471,6.676471,3.970588,...,10.352941,24.0,9.0,75.242424,35.939394,14.424242,7.393939,4.272727,18.151515,12.545455
853,2018.0,1276.0,1260.0,27.0,7.0,73.647059,33.323529,14.588235,6.176471,3.088235,...,9.176471,27.0,5.0,71.687500,31.875000,15.437500,6.656250,2.375000,14.000000,12.375000
854,2018.0,1437.0,1242.0,30.0,4.0,87.058824,35.588235,16.676471,6.676471,3.970588,...,10.352941,27.0,7.0,81.500000,35.205882,17.000000,6.617647,4.176471,15.294118,11.676471


In [13]:
reg_2022_df = pd.read_csv("./2022RegularSeason.csv",
                    names=['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc', 'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF'],
                    #dtype={'Season': 'str', 'DayNum': 'str', 'WTeamID': 'str', 'WScore': 'int', 'LTeamID': 'str', 'LScore': 'int', 'WLoc': 'str', 'NumOT': 'str', 'WFGM': 'int', 'WFGA': 'int', 'WFGM3': 'int', 'WFGA3': 'int', 'WFTM': 'int', 'WFTA': 'int', 'WOR': 'int', 'WDR': 'int', 'WAst': 'int', 'WTO': 'int', 'WStl': 'int', 'WBlk': 'int', 'WPF': 'int', 'LFGM': 'int', 'LFGA': 'int', 'LFGM3': 'int', 'LFGA3': 'int', 'LFTM': 'int', 'LFTA': 'int', 'LOR': 'int', 'LDR': 'int', 'LAst': 'int', 'LTO': 'int', 'LStl': 'int', 'LBlk': 'int', 'LPF': 'int'},
                    encoding = "ISO-8859-1")
reg_2022_df = reg_2022_df.drop(labels=0, axis=0)
tourney_team_ids = [1211, 1112,1242,1124, 1120, 1246,1437, 1181, 1458, 1397,1345,1403,1417, 1228, 1344, 1116, 1163, 1222, 1388, 1234, 1104, 1261, 1400, 1161, 1425, 1293,1277, 1326, 1129, 1314, 1361,1371, 1166, 1395, 1266, 1272, 1362, 1274, 1260, 1172,1235, 1276,1461,1353,1231, 1439, 1323,1412,1350,1308,1151, 1355, 1436,1103, 1255,1463, 1159, 1286, 1174,1389,1240,1168,1209, 1313, 1460, 1136,1411, 1394]
team_avgs_2022 = pd.DataFrame(columns=cols)

for key in tourney_team_ids:
    team_wins = reg_2022_df[reg_2022_df['WTeamID'] == str(key)]
    team_losses = reg_2022_df[reg_2022_df['LTeamID'] == str(key)]
    
    team_win_df = team_wins[['Season', 'WTeamID', 'WScore', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']].apply(pd.to_numeric)
    team_loss_df = team_losses[['Season', 'LTeamID', 'LScore', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']].apply(pd.to_numeric)

    w_sum = team_win_df.sum(axis=0, numeric_only=True)

    l_sum = {'Season': 0, 'LTeamID': 0, 'LScore': 0, 'LFGM': 0, 'LFGA': 0, 'LFGM3': 0, 'LFGA3': 0, 'LFTM': 0, 'LFTA': 0, 'LOR': 0, 'LDR': 0, 'LAst': 0, 'LTO': 0, 'LStl': 0, 'LBlk': 0, 'LPF': 0}
    if not team_losses.empty:
        l_sum = team_loss_df.sum(axis=0, numeric_only=True)

    tot_games = (len(team_win_df.index) + len(team_loss_df.index))
    season_total = (int(w_sum['Season']) + int(l_sum['Season'])) / tot_games
    id_total = (w_sum['WTeamID'] + l_sum['LTeamID']) / tot_games
    score_total = (w_sum['WScore'] + l_sum['LScore']) / tot_games
    rebs_total = (w_sum['WOR'] + l_sum['LOR'] + w_sum['WDR'] + l_sum['LDR']) / tot_games
    ast_total = (w_sum['WAst'] + l_sum['LAst']) / tot_games
    stl_total = (w_sum['WStl'] + l_sum['LStl']) / tot_games
    blk_total = (w_sum['WBlk'] + l_sum['LBlk']) / tot_games
    pf_total = (w_sum['WPF'] + l_sum['LPF']) / tot_games
    to_total = (w_sum['WTO'] + l_sum['LTO']) / tot_games
       
    team_avgs_2022.loc[len(team_avgs_2022.index)] = [season_total, id_total, len(team_win_df.index), len(team_loss_df.index), score_total, rebs_total, ast_total, stl_total, blk_total, pf_total, to_total]
team_avgs_2022


,Season,TeamID,Wins,Losses,PPG,RPG,APG,SPG,BPG,PFPG,TOPG
0,2022.0,1211.0,26.0,3.0,87.827586,41.482759,18.206897,6.655172,5.862069,15.413793,11.758621
1,2022.0,1112.0,31.0,3.0,84.558824,41.352941,19.911765,6.705882,5.705882,16.470588,13.176471
2,2022.0,1242.0,28.0,6.0,78.588235,37.352941,15.441176,6.411765,4.147059,15.911765,12.500000
3,2022.0,1124.0,26.0,6.0,76.500000,37.125000,15.843750,8.812500,3.406250,15.843750,12.500000
4,2022.0,1120.0,27.0,5.0,78.718750,39.937500,14.593750,8.781250,7.937500,18.562500,12.093750
...,...,...,...,...,...,...,...,...,...,...,...
63,2022.0,1313.0,24.0,6.0,75.100000,38.766667,12.633333,6.700000,3.666667,17.666667,14.000000
64,2022.0,1460.0,21.0,13.0,75.470588,34.882353,13.941176,5.764706,2.941176,14.382353,12.294118
65,2022.0,1136.0,22.0,9.0,77.903226,41.548387,14.096774,6.322581,4.451613,16.806452,13.903226
66,2022.0,1411.0,18.0,12.0,69.233333,39.533333,10.833333,5.566667,5.000000,17.800000,14.933333


In [ ]:
#d = {'Team1ID': [1211,1129,1163,1116,1104,1403,1277,1181,1124,1314,1388,1417,1400,1345     1211,1308,1323,1277,1124,], 'Team2ID': [1209,1272,1308,1436,1323,1268,1172,1168,1313,1266,1231,1103,1439,1463,       1272,1116,1403, 1181]}
#temp_df = pd.DataFrame(data=d)

In [6]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(hidden_layer_sizes=200, max_iter=200).fit(train_data, train_labels)
predictions = model.predict(test_data)
predictions

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])